In [73]:
import numpy as np
import pandas as pd
import scipy
from scipy import integrate
import math
import matplotlib.pyplot as plt
import tqdm
%matplotlib inline  


Начальные данные

In [2]:
# радиус барабана
Rb = 10

# радиус объекта
Ro = 1

# высота объекта
Ho = 10

# угловая скорость барабана
Wb = 0.175                    # рад/с ~10 град в сек

# угловая скорость объекта
Wo = 0.35                   # рад/с ~10 град в сек

# расстояние до мишени
L = 20

# ширина мишени
Wm = 8

# высота мишени
Hm = 15


In [3]:
starting_point_b = np.array([0, Rb])
starting_point_o = np.array([0, Ro])

In [4]:
# angle in radians
def rotate_point_0 (point, angle):
    x1 = point[0] * np.cos(angle) - point[1] * np.sin(angle)
    y1 = point[0] * np.sin(angle) + point[1] * np.cos(angle)
    return np.array([x1, y1])

In [5]:
point = np.array([1, 2])

In [6]:
angle = 90
angle_rad = np.deg2rad(angle)
angle_rad


1.5707963267948966

In [7]:
rotate_point_0 (point, angle_rad)

array([-2.,  1.])

In [8]:
# угол поворота, w - угловая скорость
def get_current_angle (w, t):
    return (w*t) % (2*np.pi)

In [9]:
def get_object_center (Wb, point_b, t):
    alpha_b = get_current_angle(Wb, t)
    vec_b = rotate_point_0(point_b, alpha_b)
    return vec_b

In [10]:
# point_b  начальное опложение центра объекта, point_o - начальное положение точки на объекте относительно его центра
def get_point_coords (Wb, Wo, point_b, point_o, t):
    vec_b = get_object_center(Wb, point_b, t)
    alpha_o = get_current_angle(Wo, t)
    vec_o = rotate_point_0(point_o, alpha_o)
    return vec_b + vec_o

In [11]:
def get_normal_vec(center, point):
    return point - center

In [12]:
time_spent = 1

In [13]:
cent = get_object_center(Wb, starting_point_b, time_spent)
cent

array([-1.74108138,  9.84726539])

In [14]:
point = get_point_coords(Wb, Wo, starting_point_b, starting_point_o, time_spent)
point

array([-2.08397918, 10.7866381 ])

In [15]:
normal = get_normal_vec(cent, point)
normal

array([-0.34289781,  0.93937271])

In [16]:
def get_cosin(v1, v2):
    return 1 - scipy.spatial.distance.cosine(v1, v2)

In [17]:
def get_sin(v1, v2):
    return math.sqrt(1 - get_cosin(v1, v2)**2)

In [18]:
get_cosin([1, 0], [0.25, -1])

0.24253562503633297

In [19]:
get_sin([1, 0], [0.25, -1])

0.9701425001453319

In [20]:
# f= lambda y, x: x*y**2 + 1
# integrate.dblquad(f, 6, 9, lambda x: -1, lambda x: x+1)

In [71]:
def get_function_value (target_p, object_p, n):

    
    n1 = np.array([0, 0, -1])   # нормаль к плоскости мишени
    s = target_p - object_p     # вектор он точки на объекте к точке на мишени
    s1 = object_p - target_p 
    cos_teta = get_cosin(n, s)
    
#     print ("target=", target_p)
#     print ("object=", object_p)
#     print ("normal=", n)
#     print ("s=", s)
#     print ("\n")
    
    
#     if (cos_teta > 0):
#         print ("reachable")
    if cos_teta < 0:            # точка на объекте не видима с точки на мишени
        return 0
    
    cos_teta1 = abs(get_cosin(n1, s1))
    
    sin_teta1 = abs(get_sin(n1, s1))
    res = (sin_teta1 * cos_teta)
    if (res < 0):
        print ("Achtung! res = ", res)
    return res # * Im0

# point - точка на цилиндре, n - нормаль к этой точке
# t1 - верхняя левая точна мишени t2 - нижняя правая точка мишени (мишень - плоская)
def integrate_one_point(point, n, t1, t2):
    x_target = t1[0]
    f= lambda y, z: get_function_value (np.array([x_target, y, z]), point, n)
    (val, err) = integrate.dblquad(f, t1[1], t2[1], lambda z: t1[2], lambda z: t2[2], epsabs = 1e-2, epsrel=1e-2)
#     print (val, err)
    return val

In [22]:
integrate_one_point (np.array([50, -25, 0]), np.array([0, -1, 0]), np.array([150, 10, 20]), np.array([150, -10, -20]))

0.0 0


0.0

In [23]:
def get_target_coords (L, Wm, Hm):
    t1 = np.array([L, Wm/2, Hm/2])
    t2 = np.array([L, -Wm/2, -Hm/2])
    return (t1, t2)
    

In [24]:
# point_o - начальное положение точки на объекте относительно его центра
def get_point_coords (point_center, Wo, start_angle, t, Ro):
    alpha_o = (get_current_angle(Wo, t) + start_angle) % (2*np.pi)
    point_o = np.array([0, Ro])
    vec_o = rotate_point_0(point_o, alpha_o)
    return point_center + vec_o

In [25]:
def get_object_points(point_center, num, Wo, t, Ro):
    res = []
    delta = (2*np.pi) / (num)
    for i in range (0, num):
        p = get_point_coords (point_center, Wo, delta*i, t, Ro)
        res.append([p[0], p[1], 0])
    return res

In [26]:
#get_object_points([0, 0], 12, 0.175, 0, Ro)

In [27]:
turn_time = 6.283/ abs(Wb) # время полного оборота барабана

In [28]:
turn_time

35.902857142857144

In [81]:
time_n = 50    # количество измерений за 1 оборот

In [82]:
time_delta = turn_time / time_n
time_delta

0.7180571428571428

In [83]:
turns_num = 5    # количество оборотов барабана

In [84]:
calc_num = time_n * turns_num
calc_num

250

In [ ]:
cur_time = 0
obj_point_num = 24      # колчество точек на цилиндре, в которых вычисляется толщина

res = np.zeros(obj_point_num)
t1, t2 = get_target_coords (L, Wm, Hm)

for i in tqdm.notebook.tqdm(range (0, calc_num)):
#     print ("\nturn\n")
    cent = get_object_center(Wb, starting_point_b, cur_time)
    cent_3d = np.array([cent[0], cent[1], 0])
    pts = get_object_points(cent, obj_point_num, Wo, cur_time, Ro)
    
    for i in range (0, obj_point_num):
        p = pts[i]
        normal = get_normal_vec(cent_3d, p)
        normal_3d = np.array([normal[0], normal[1], 0])
        
        res[i] = res[i] + time_delta * integrate_one_point(pts[i], normal_3d, t1, t2)
    
    cur_time += time_delta

    
res

In [ ]:
plt.figure()
plt.plot(res)
plt.ylim(bottom = 0)
plt.show()